### Necessary Imports and Settings
    for any step in the following notebook

In [17]:
import os
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import cmb_anomaly_utils as cau

In [7]:
max_sim_num                     = 1000
base_path                       = "./output/noisy_masked_measure/"
cmb_series                      = "commander"
run_inputs  = cau.run_utils.RunInputs()
run_inputs.mask_fpath           = "./input/cmb_fits_files/COM_Mask_CMB-common-Mask-Int_2048_R3.00.fits"
run_inputs.cmb_fpath            = "./input/cmb_fits_files/COM_CMB_IQU-{}_2048_R3.00_full.fits".format(cmb_series)
# run_inputs.cmb_dir_anom_fpath   = "./output/cmb_inpainted_all_dir_cap_anom.txt"
run_inputs.sims_path            = "./input/{}_sims/".format(cmb_series)
run_inputs.noise_path           = "./input/{}_noise/".format(cmb_series)
# run_inputs.sims_dir_anom_path   = "./output/sims_inpainted_all_dir_cap_anom_5deg/"
run_inputs.geom_flag            = cau.const.CAP_FLAG
run_inputs.measure_flag         = cau.const.VAR_FLAG
run_inputs.is_masked            = True
run_inputs.min_pix_ratio        = 0.1
run_inputs.nside                = 64
run_inputs.dir_nside            = 16
run_inputs.geom_start           = 0
run_inputs.geom_stop            = 180
run_inputs.delta_geom_samples   = 1
run_inputs.stripe_thickness     = 20

dir_search_cap_sizes = cau.stat_utils.get_range(5, 90, 5)

### Pepare Temperature Maps

In [9]:
print("Warming Up!")

map_gen = cau.run_utils.MapGenerator(**run_inputs.to_kwargs())

dummy_map = map_gen.create_dummy_map()
dir_lat, dir_lon = cau.coords.get_healpix_latlon(run_inputs.dir_nside)
ndir = cau.coords.get_npix(run_inputs.dir_nside)
measure_dir_indices = np.arange(ndir, dtype=int)

cmb_measure = np.zeros(ndir)
sims_measure = np.zeros((max_sim_num, ndir))

print("Reading CMB Data:")
cmb_map = map_gen.create_cmb_map()
cau.map_utils.remove_monopole_dipole(cmb_map)
print("\t-Done-")

print("Providing Simulations:")
sims_maps = []
for sim_num in range(max_sim_num):
    print("\t{:04}/{:04}\r".format(sim_num + 1, max_sim_num), end="")
    sim_pix_map = map_gen.create_sim_map_from_txt(sim_num, use_noise=False)
    cau.map_utils.remove_monopole_dipole(sim_pix_map)
    sims_maps.append(sim_pix_map)
print("")

Warming Up!
Reading CMB Data:
	-Done-
Providing Simulations:
	1000/1000


### Calculate Cap (RAW)Variances
    "Pepare Temperature Maps" Section is needed in memory

In [5]:
for cap_size in dir_search_cap_sizes:
    print("Disc Size(deg): {:2}".format(cap_size))
    print("Calculating Cap Measures:")
    for dir_i in measure_dir_indices:
        print(f"\t{dir_i}/{ndir - 1}\r", end="")
        # Separating Common Geometry -> Top cap & Bottom cap
        plat, plon = dir_lat[dir_i], dir_lon[dir_i]
        dummy_map.change_pole(plat, plon)
        top_filter, bottom_filter = cau.geometry.get_top_bottom_caps_selection_filters( pix_map = dummy_map,
                                                                                        cap_angle = cap_size)
        top_cap, bottom_cap = cau.geometry.get_top_bottom_caps_by_filters( pix_map = cmb_map,
                                                                           top_sel = top_filter,
                                                                           bottom_sel = bottom_filter)
        visible_ratio = top_cap.get_visible_pixels_ratio()
        # CMB measure
        cmb_measure[dir_i] = \
            np.nan if visible_ratio < run_inputs.min_pix_ratio else \
                cau.measure.calc_var(top_cap, bottom_cap, **run_inputs.to_kwargs())
        # Simulation Measure
        for sim_num in range(max_sim_num):
            top_cap, bottom_cap = cau.geometry.get_top_bottom_caps_by_filters( pix_map = sims_maps[sim_num],
                                                                               top_sel = top_filter,
                                                                               bottom_sel = bottom_filter)
            sims_measure[sim_num, dir_i] = \
                np.nan if visible_ratio < run_inputs.min_pix_ratio else \
                    cau.measure.calc_var(top_cap, bottom_cap, **run_inputs.to_kwargs())
    print("")

    np.savetxt(base_path + "{}_sims_var_{:02}cap.txt".format(cmb_series, cap_size), sims_measure)
    np.savetxt(base_path + "{}_cmb_var_{:02}cap.txt".format(cmb_series, cap_size), cmb_measure)

# print("-Done-")

Disc Size(deg): 5.0
Calculating Cap Measures:
	3071/3071
Disc Size(deg): 10.0
Calculating Cap Measures:
	3071/3071
Disc Size(deg): 15.0
Calculating Cap Measures:
	3071/3071
Disc Size(deg): 20.0
Calculating Cap Measures:
	3071/3071
Disc Size(deg): 25.0
Calculating Cap Measures:
	3071/3071
Disc Size(deg): 30.0
Calculating Cap Measures:
	3071/3071
Disc Size(deg): 35.0
Calculating Cap Measures:
	3071/3071
Disc Size(deg): 40.0
Calculating Cap Measures:
	3071/3071
Disc Size(deg): 45.0
Calculating Cap Measures:
	3071/3071
Disc Size(deg): 50.0
Calculating Cap Measures:
	3071/3071
Disc Size(deg): 55.0
Calculating Cap Measures:
	3071/3071
Disc Size(deg): 60.0
Calculating Cap Measures:
	3071/3071
Disc Size(deg): 65.0
Calculating Cap Measures:
	3071/3071
Disc Size(deg): 70.0
Calculating Cap Measures:
	3071/3071
Disc Size(deg): 75.0
Calculating Cap Measures:
	3071/3071
Disc Size(deg): 80.0
Calculating Cap Measures:
	3071/3071
Disc Size(deg): 85.0
Calculating Cap Measures:
	3071/3071
Disc Size(deg):

### Finding Dipole Direction on Cap-Var Maps Considering $Mean(Var) \; \& \; Var(Var)$
    "Calculate Cap (RAW)Variances" Section has to be computed and stored

In [4]:
n_cap_sizes = len(dir_search_cap_sizes)

# The number of the direction's pixel will be stored
cmb_directions = np.zeros((n_cap_sizes))
sims_directions = np.zeros((n_cap_sizes, max_sim_num))

dummy_map = cau.dtypes.PixMap.create_dummy(run_inputs.dir_nside)

for cs_i, cap_size in enumerate(dir_search_cap_sizes):
    cmb_var  = np.loadtxt(base_path + "{}_cmb_var_{:02}cap.txt".format(cmb_series, cap_size))
    sims_var = np.loadtxt(base_path + "{}_sims_var_{:02}cap.txt".format(cmb_series, cap_size))

    #----COMMON ASSETS----
    # Common Mean and Var of Simulated Maps
    _mean = np.mean(sims_var, axis = 0)
    _var  = np.var(sims_var, axis = 0)
    # Common Mask
    _mask = np.isnan(cmb_var)
    dummy_map.mask = _mask if np.any(_mask) else None
    
    #----SPECIFIC CALCULATIONS----
    # CMB Direction
    dummy_map.data = (cmb_var - _mean) / _var
    cmb_directions[cs_i] = cau.map_utils.get_dipole_direction_index(pix_map = dummy_map,
                                                                    dir_nside = run_inputs.dir_nside)
    # Simulations Directions
    for sim_num in range(max_sim_num):
        dummy_map.data = (sims_var[sim_num] - _mean) / _var
        sims_directions[cs_i, sim_num] = \
            cau.map_utils.get_dipole_direction_index(pix_map = dummy_map,
                                                     dir_nside = run_inputs.dir_nside)

np.savetxt(base_path + "{}_cmb_var_dir.txt".format(cmb_series), cmb_directions, fmt = '%i')
np.savetxt(base_path + "{}_sims_var_dir.txt".format(cmb_series), sims_directions, fmt = '%i')

### Taking Stripes and Calculating $Var(T)$ Considering $Mean(Var) \; \& \; Var(Var)$
    "Pepare Temperature Maps" Section is needed in memory

In this section, to compute measures for each map we should:

1. take the direction of the target map (it can be CMB or any simulated map) and align all simulated maps to the direction of the target map
2. compute measure for stripes in target map and all simulated maps
3. subtract the mean of stripes of simulated maps from target map and divide it by the variance of the measure 

In [ ]:
cmb_directions  = np.loadtxt(base_path + "{}_cmb_var_dir.txt".format(cmb_series))
sims_directions = np.loadtxt(base_path + "{}_sims_var_dir.txt".format(cmb_series))

n_cap_sizes = len(dir_search_cap_sizes)
dir_lat, dir_lon = cau.coords.get_healpix_latlon(run_inputs.dir_nside)

cmb_stripe_measures  = np.zeros((n_cap_sizes, len(run_inputs.geom_range)))
sims_stripe_measures = np.zeros((n_cap_sizes, sim_num, len(run_inputs.geom_range)))

aux_measures = np.zeros((max_sim_num, len(run_inputs.geom_range)))

# Handy function with global values
def calc_subtract_mean_field(_measure):
    for sim_num in range(max_sim_num):
        dummy_map.raw_data = sims_maps[sim_num].raw_data
        aux_measures[sim_num] = cau.measure.get_measure(dummy_map, **run_inputs.to_kwargs())
    _mean  = np.mean(aux_measures, axis = 0)
    _var   = np.var(aux_measures, axis = 0)
    return (_measure - _mean) / _var

#TODO: unify the geometry!
print("Computing stripe measures (subtracting stripes' mean fields)")
for cs_i, cap_size in enumerate(dir_search_cap_sizes):
    print("Disc Size(deg): {:2}".format(cap_size))
    #----CMB Measure----
    print("Computing CMB")
    dir_index = cmb_directions[cs_i]
    plat, plon = dir_lat[cs_i], dir_lon[cs_i]
    dummy_map.change_pole(plat, plon)
    dummy_map.raw_data = cmb_map.raw_data
    _measure = cau.measure.get_measure(dummy_map, **run_inputs.to_kwargs())
    cmb_stripe_measures[cs_i] = calc_subtract_mean_field(_measure)
    print("\t-Done-")

    #----Simulation Measure----
    print("Computing Simulations")
    for sim_num in range(max_sim_num):
        print("\t{:04}/{:04}\r".format(sim_num + 1, max_sim_num), end="")
        dir_index = sims_directions[cs_i, sim_num]
        plat, plon = dir_lat[cs_i], dir_lon[cs_i]
        dummy_map.change_pole(plat, plon)
        dummy_map.raw_data = sims_maps[sim_num].raw_data
        _measure = cau.measure.get_measure(dummy_map, **run_inputs.to_kwargs())
        sims_stripe_measures[cs_i, sim_num] = calc_subtract_mean_field(_measure)
    print("\t-Done-")
    
np.savetxt(base_path + "{}_cmb_stripe_var_measure.txt".format(cmb_series), cmb_stripe_measures)
np.savetxt(base_path + "{}_sims_stripe_var_measure.txt".format(cmb_series), sims_stripe_measures)